In [1]:
import os, glob, json, pickle
from openeye import *

In [2]:
#Extract the mol2_gaff files
os.system('tar -xvzf ../mol2files_gaff.tar.gz')
#Extract the mol2_sybyl files
os.system('tar -xvzf ../mol2files_sybyl.tar.gz')

0

In [3]:
#Load in our input files
gaff_mol2s = sorted(glob.glob('mol2files_gaff/*.mol2'))
sybyl_mol2s = sorted(glob.glob('mol2files_sybyl/*.mol2'))
dbfile = open('../database.json')
db = json.load(dbfile)

In [4]:
def concatTagOEMol(mol2files, outfname):
    
    ofs = oemolostream()
    if not ofs.open(outfname):
        OEThrow.Fatal("Unable to open %s for writing" % outfname)
    
    for mol2 in mol2files:
        fname = mol2.split('/')[-1]
        freesolvid = fname.split('.mol2')[0]

        with oemolistream(mol2) as ifs:
            for imol in ifs.GetOEGraphMols():
                # Add generic tag
                imol.SetData(OEGetTag('FreeSolvID'), freesolvid)
                # Attach as SD Data
                OESetSDData(imol, 'FreeSolvID', str(freesolvid))
                
                for k,v in sorted(db[freesolvid].items()):
                    # SD Data is read in as a CSV
                    # Put nicknames/IUPAC names into a list.
                    if 'iupac' == k:
                        v = str(  [v.strip()]  )
                    elif 'nickname' == k:
                        v = str(  [v.strip()]  )
                    OESetSDData(imol, k, str(v) )
                OEWriteConstMolecule(ofs, imol)        
    ofs.close()
    
    # Check that our database is unchanged
    with oemolistream(outfname) as ifs:
        for imol in ifs.GetOEGraphMols():
            freesolvid = imol.GetData('FreeSolvID')

            reconst_data = {}
            for dp in OEGetSDDataPairs(imol):
                reconst_data[dp.GetTag()] = dp.GetValue()
            #print(reconst_data)
        
            # Check str matches since SD data are strs
            for k,v in db[freesolvid].items():
                if  k in ['iupac', 'nickname', 'FreeSolvID']:
                    pass
                else:
                    assert(reconst_data[k] == str(v))

In [5]:
concatTagOEMol(gaff_mol2s, 'freesolv_gaff.oeb.gz')
concatTagOEMol(sybyl_mol2s, 'freesolv_sybyl.oeb.gz')